# Exercise: Image classification with medical image

In [ ]:
# 安裝所需套件
!pip install -q torchio
!pip install -q transformers==4.30.0 datasets evaluate accelerate

## Import packages

In [ ]:
# 匯入基本操作相關套件
import torchio as tio
import glob
import numpy as np
import matplotlib.pyplot as plt

# 匯入 PyTorch 相關套件
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## Load dataset (chest x-ray)

#### Curated Chest X-Ray Image Dataset for COVID-19
* resource: http://dx.doi.org/10.17632/9xkhgts2s6.1
* dataset_url: https://github.com/TA-aiacademy/CMU_Course/releases/download/image_classification_data/Curated_Chest_X-Ray.zip

In [ ]:
!wget https://github.com/TA-aiacademy/CMU_Course/releases/download/image_classification_data/Curated_Chest_X-Ray.zip
!unzip -q Curated_Chest_X-Ray.zip

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Curated_Chest_X-Ray/")

In [ ]:
dataset

In [ ]:
train_dataset = dataset['train']
valid_dataset = dataset['validation']

In [ ]:
item = next(iter(train_dataset))

In [ ]:
print(item['label'])
item['image']

In [ ]:
item['image'].size

In [ ]:
train_dataset.features["label"].names

In [ ]:
labels = train_dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [ ]:
labels

現在，按照標籤 id 轉換成名稱：

In [ ]:
id2label[1]

## Preprocess


接下來的步驟是載入指定模型使用的影像處理器，將影像處理成張量：

In [ ]:
from transformers import AutoImageProcessor

# checkpoint = "google/vit-base-patch16-224-in21k"  # model name
# checkpoint = "google/efficientnet-b6"
checkpoint = "microsoft/cvt-13"
# checkpoint = "google/mobilenet_v2_1.0_224"
# checkpoint = "microsoft/resnet-50"
# checkpoint = "facebook/convnext-tiny-224"
# checkpoint = "facebook/convnext-base-224"
# # checkpoint = "facebook/convnext-large-224"

image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
image_processor

---
將影像進行轉換，使模型更具一般性以應付過擬合的情況。這裡會使用的 torchvision 中 transforms 的模組，但也能替換成其他適用的影像處理套件。

將其調整影像大小以及隨機仿射處理，並使用影像的平均值和標準差進行標準化：

In [ ]:
from torchvision.transforms import Resize, RandomAffine, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
# size = (image_processor.crop_size['height'], image_processor.crop_size['width'])

size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([Resize(size),
#                        RandomHorizontalFlip(),
                       RandomAffine(degrees=10, scale=(0.9, 1.1)),
                       ToTensor(),
                       normalize])

接下來創建一個預處理函數，轉換並回傳影像的像素值作為模型的輸入：

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

要在整個資料集上應用預處理函數，可以使用 Hugging Face 資料集的 [with_transform](https://huggingface.co/docs/datasets/v2.11.0/en/package_reference/main_classes#datasets.Dataset.with_transform) 方法。當載入資料集的一個元素時，轉換會即時套用：

In [ ]:
train_ds = train_dataset.with_transform(transforms)
valid_ds = valid_dataset.with_transform(transforms)

In [ ]:
import matplotlib.pyplot as plt
import torch

In [ ]:
item = next(iter(train_ds))

In [ ]:
item

In [ ]:
print(item['label'])
print(item['pixel_values'].size())
plt.imshow(torch.permute(item['pixel_values'], (1, 2, 0)))

現在使用 [DefaultDataCollator](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/data_collator#transformers.DefaultDataCollator) 創建一個批次樣本。與 Hugging face 裡 Transformers 的其他資料收集器不同，DefaultDataCollator 不會套用額外的預處理，例如填充（padding）。

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

## Evaluate

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

然後創建一個函數，將預測及標籤使用 [compute](https://huggingface.co/docs/evaluate/v0.4.0/en/package_reference/main_classes#evaluate.EvaluationModule.compute) 以計算準確度：

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Build model

現在已準備好開始訓練模型了！使用 [AutoModelForImageClassification](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/auto#transformers.AutoModelForImageClassification) 載入模型。指定標籤的數量以及標籤的對應方式：

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

In [ ]:
# freeze layers without training
for param in model.cvt.encoder.stages[:1].parameters():
    param.requires_grad = False

## Train model

接著的階段，只剩以下三個步驟：

1. 在 [TrainingArguments](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments) 中定義訓練的超參數。請務必留意資料集中未使用的資訊，設定 remove_unused_columns=False 可以防止被刪除未使用到的資訊！例如 image，這會導致無法獲得 pixel_values。另一個必需設定的參數是 output_dir，指定模型儲存的位置。通過設定 push_to_hub=True 將模型上傳至 Hub（需要登入 Hugging Face 才能上傳模型）。在每個 epoch 結束時，[Trainer](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer) 將評估準確性並儲存訓練模型。
2. 將訓練參數、模型、資料集、預處理器、資料收集器以及計算評估指標函數傳遞給 [Trainer](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer)。
3. 呼叫 [train](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer.train) 來微調模型。

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="my_model_exercise",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy")

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

# Inference

現在，微調後的模型以存放在指定路徑，並可使用它來進行推論！

載入想要進行推論的影像：

In [ ]:
test_dataset = dataset['test']
image = test_dataset['image'][1]
label = test_dataset['label'][1]

In [ ]:
label

In [ ]:
id2label[label]

In [ ]:
image

使用微調後的模型進行推論最簡單的方法是在 pipline() 中設定。藉由指定的模型建構一個影像分類的 pipeline，然後將影像傳遞給它：

In [ ]:
from transformers import pipeline

classifier = pipeline("image-classification", model="my_model_exercise/checkpoint-500")
classifier(image)

將輸入傳遞給模型，並回傳 logits（尚未經過 softmax）：

載入影像處理器對影像進行預處理，並以 PyTorch 的張量型態回傳作為輸入：

In [ ]:
from transformers import AutoImageProcessor
import torch

image_processor = AutoImageProcessor.from_pretrained("my_model_exercise/checkpoint-500")
inputs = image_processor(image.convert('RGB'), return_tensors="pt")

In [ ]:
inputs

將輸入傳遞給模型，並回傳 logits（尚未經過 softmax）：

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("my_model_exercise/checkpoint-500")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]

In [ ]:
model.config.id2label[label]